# 因子高低价统计分布
1. 读取本地数据
2. 添加signal，添加high,low获取signal_data
3. 对最高价与最低价进行分析,输出做多的Quantile75的止盈止损位

## 1_读取本地数据

In [1]:
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings

warnings.filterwarnings("ignore")
dataview_folder = 'JAQS_Data/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

D:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Dataview loaded successfully.


In [17]:
def mask_index_member():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = ~(df_index_member ==0) #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
    # 跌停
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [18]:
mask = mask_index_member()
can_enter,can_exit = limit_up_down()
price = dv.get_ts('close_adj')
high = dv.get_ts('high_adj')
low = dv.get_ts('low_adj')
price_bench = dv.data_benchmark

Add formula failed: name [up_limit] exist. Try another name.
Add formula failed: name [down_limit] exist. Try another name.


## 2_添加signal，添加high,low获取signal_data

In [19]:
dv.add_formula("Cor_Vwap",'- Correlation(vwap_adj, volume, 10)',is_quarterly=False)
signal = dv.get_ts("Cor_Vwap")

Add formula failed: name [factor] exist. Try another name.


In [27]:
sig = SignalDigger(output_format=None)
sig.process_signal_before_analysis(signal,
                                   price=price,
                                   high=high,
                                   low=low,
                                   n_quantiles=5,
                                   mask=mask,
                                   can_enter=can_enter,
                                   can_exit=can_exit,
                                   period=5,
                                   #benchmark_price=price_bench,
                                   forward=True,)
signal_data = sig.signal_data
print(signal_data.head())

Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
                        signal    return  upside_ret  downside_ret  quantile
trade_date symbol                                                           
20120105   000008.SZ  0.244504  0.111011    0.133058     -0.038595         5
           000031.SZ  0.175844  0.073580    0.101128     -0.014574         4
           000046.SZ -0.655705  0.023770    0.067996     -0.027827         2
           000400.SZ  0.413553  0.027469    0.052792     -0.087961         5
           000413.SZ -0.770937  0.139239    0.170798     -0.014607         1


## 3_对最高价与最低价进行分析,输出做多的Quantile75的止盈止损位

In [20]:
from jaqs.research import analysis
result = analysis(signal_data=signal_data,
                  is_event=False,
                  period=5)

In [25]:
print('Percentile75止盈:', result["space"].loc['upside_space_percentile75','long_space'])

Percentile75止盈: 0.0512136099887


In [26]:
print('Percentile75止损:', result["space"].loc['downside_space_percentile75','long_space'])

Percentile75止损: -0.0196126635889
